<a href="https://colab.research.google.com/github/gargi411/Optimized-weed-detection-using-bio-inspired-genetic-algorithm-and-deep-CNN/blob/main/Weed_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: install + imports
!pip -q install tensorflow-datasets

import os, math, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import classification_report, confusion_matrix
import itertools
print("TF:", tf.__version__)


TF: 2.19.0


In [ ]:
# Cell 2: load dataset (will download ~5 GB)
ds, ds_info = tfds.load("deep_weeds", split=['train'], with_info=True, as_supervised=True)
# deep_weeds is delivered as a single 'train' split; we'll create train/val/test ourselves
dataset = ds[0]
print(ds_info)
CLASS_NAMES = ds_info.features['label'].names
print("Classes:", CLASS_NAMES)


tfds.core.DatasetInfo(
    name='deep_weeds',
    full_name='deep_weeds/3.0.0',
    description="""
    The DeepWeeds dataset consists of 17,509 images capturing eight different weed species native to Australia in situ with neighbouring flora.The selected weed species are local to pastoral grasslands across the state of Queensland.The images were collected from weed infestations at the following sites across Queensland: "Black River", "Charters Towers",  "Cluden", "Douglas", "Hervey Range", "Kelso", "McKinlay" and "Paluma".
    """,
    homepage='https://github.com/AlexOlsen/DeepWeeds',
    data_dir='/root/tensorflow_datasets/deep_weeds/3.0.0',
    file_format=tfrecord,
    download_size=469.32 MiB,
    dataset_size=469.99 MiB,
    features=FeaturesDict({
        'image': Image(shape=(256, 256, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=9),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False

In [ ]:
# Cell 3: preprocessing + split
IMG_SIZE = 224
BATCH = 32
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = dataset.map(preprocess, num_parallel_calls=AUTOTUNE)

# shuffle then split 80/10/10
ds_size = 0
for _ in dataset:
    ds_size += 1
print("Total images:", ds_size)

dataset = dataset.shuffle(20000, seed=SEED, reshuffle_each_iteration=False)
train_size = int(0.8 * ds_size)
val_size   = int(0.1 * ds_size)
test_size  = ds_size - train_size - val_size

train_ds = dataset.take(train_size).batch(BATCH).prefetch(AUTOTUNE)
rest = dataset.skip(train_size)
val_ds = rest.take(val_size).batch(BATCH).prefetch(AUTOTUNE)
test_ds = rest.skip(val_size).batch(BATCH).prefetch(AUTOTUNE)

print("Split sizes:", train_size, val_size, test_size)


Total images: 17509
Split sizes: 14007 1750 1752


In [ ]:
# Cell 4: model (transfer learning)
base = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3),
                                         include_top=False, weights='imagenet')
base.trainable = False  # freeze base for speed

inputs = tf.keras.Input(shape=(IMG_SIZE,IMG_SIZE,3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 9)              │        11,529 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,269,513 (8.66 MB)

 Trainable params: 11,529 (45.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Cell 5: train (use modest epochs now; enlarge if time allows)
EPOCHS = 10   # start small to test; increase to 25-50 if time permits
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_deepweeds_mobilenet.h5", save_best_only=True, monitor='val_sparse_categorical_accuracy', mode='max'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=6, restore_best_weights=True, mode='max', verbose=1)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)


Epoch 1/10
 68/438 ━━━━━━━━━━━━━━━━━━━━ 8:41 1s/step - loss: 2.3586 - sparse_categorical_accuracy: 0.1636

In [ ]:
# Cell 6: evaluate & plots
# Evaluate on test set
loss, acc = model.evaluate(test_ds)
print(f"Test acc: {acc:.4f}")

# Predictions for classification report / confusion matrix
y_true = []
y_pred = []
for batch_x, batch_y in test_ds:
    probs = model.predict(batch_x)
    preds = np.argmax(probs, axis=1)
    y_true.extend(batch_y.numpy().tolist())
    y_pred.extend(preds.tolist())

print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, zero_division=0))

cm = confusion_matrix(y_true, y_pred)

# plot history
plt.figure(figsize=(6,4))
plt.plot(history.history['sparse_categorical_accuracy'], label='train_acc')
plt.plot(history.history['val_sparse_categorical_accuracy'], label='val_acc')
plt.legend(); plt.title('Accuracy'); plt.savefig('accuracy.png'); plt.show()

plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(); plt.title('Loss'); plt.savefig('loss.png'); plt.show()

# plot confusion
plt.figure(figsize=(8,6))
plt.imshow(cm, interpolation='nearest'); plt.title('Confusion matrix'); plt.colorbar()
tick_marks = np.arange(len(CLASS_NAMES))
plt.xticks(tick_marks, CLASS_NAMES, rotation=45, ha='right')
plt.yticks(tick_marks, CLASS_NAMES)
thresh = cm.max() / 2.
for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i,j], 'd'), ha='center', color='white' if cm[i,j]>thresh else 'black')
plt.tight_layout(); plt.savefig('confusion.png'); plt.show()

# Save final model (if you want the TF SavedModel)
model.save("deepweeds_mobilenet_savedmodel")
print("Saved model and plots.")
